In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import wfdb
import csv
import os
from scipy.signal import butter, filtfilt
#from PyEMD import EMD
import emd
import time
import biosppy.signals.ecg as bsp_ecg
import biosppy.signals.tools as bsp_tools
from sklearn.metrics import f1_score
from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import wfdb
import csv
import os
from scipy.signal import butter, filtfilt
import scipy
#from PyEMD import EMD
import emd
import time
from scipy.interpolate import CubicSpline
from sklearn.preprocessing import MinMaxScaler

In [29]:
"""liste_num = [fichier[:-4] for fichier in os.listdir("data") if fichier.endswith(".dat")]
for num_patient in liste_num:
    patient_record = wfdb.rdrecord(f"data/{num_patient}")
    annotation = wfdb.rdann(f'data/{num_patient}', 'atr')
    qrs = annotation.sample.tolist()
                            
    df = pd.DataFrame(patient_record.p_signal, columns= patient_record.sig_name)
    df_full = df.assign(labels= qrs + [None]*(df.shape[0]-len(qrs)))
    
    df_full.to_csv(f"data_csv/{patient_record.record_name}.csv", index=False)"""

'liste_num = [fichier[:-4] for fichier in os.listdir("data") if fichier.endswith(".dat")]\nfor num_patient in liste_num:\n    patient_record = wfdb.rdrecord(f"data/{num_patient}")\n    annotation = wfdb.rdann(f\'data/{num_patient}\', \'atr\')\n    qrs = annotation.sample.tolist()\n                            \n    df = pd.DataFrame(patient_record.p_signal, columns= patient_record.sig_name)\n    df_full = df.assign(labels= qrs + [None]*(df.shape[0]-len(qrs)))\n    \n    df_full.to_csv(f"data_csv/{patient_record.record_name}.csv", index=False)'

In [30]:
def hamilton(signal, sampling_rate, group): 
    return bsp_ecg.ecg(signal=np.array(signal), sampling_rate=sampling_rate, show=False)[2]
    rr_intervals_list = [(rr[i] - rr[i-1])/360 * 1000 for i in range(1,len(rr))]
    rr_intervals_without_outliers = remove_outliers(
        rr_intervals=rr_intervals_list,  
        low_rri=300, high_rri=2000,
        verbose= False)
    interpolated_rr_intervals = interpolate_nan_values(rr_intervals=rr_intervals_without_outliers,
                                                       interpolation_method="linear")

    nn_intervals_list = remove_ectopic_beats(rr_intervals=interpolated_rr_intervals, method="malik",verbose= False)
    interpolated_nn_intervals = interpolate_nan_values(rr_intervals=nn_intervals_list)
    return [(sum(rr[:i])/1000)*360 for i in range(1,len(rr)+1)]

In [31]:
#(signal, M, sampling_rate)
data_tab = ["mit_bih_supraventricular", "european", "mit_bih_long_term", 'mit_bih_Arrhythmia'] # , "mit_bih_noise_stress",
algo_tab = ['emd', 'hamilton']

In [32]:
df_emd = pd.DataFrame()

In [33]:
algo = "emd"

In [34]:
#data_tab = ['european']
dico_algo = {"low_comput": low_comput, 
             "emd" : full_emd, 
             "hamilton": hamilton, 
             "wavelet" : full_wavelets }
sampling_rate = 360
for data in data_tab:
    start = time.time()
    Result_DF = pd.DataFrame()

    for fichier in os.listdir(f"data_csv/{data}"):
        fs = 360
        df = pd.read_csv(f'data_csv/{data}/{fichier}')

        cible = ""
        if data in ['mit_bih_supraventricular', "mit_bih_long_term"]:
            cible = 'ECG1'
            fs = 128
            sampling_rate = 128
        elif data == "european":
            cible = "MLIII"
            fs = 250
            sampling_rate = 250
        else:
            cible = "MLII"

        if cible not in df.columns:
            continue

        signal = np.array(df[cible], dtype=np.float32) #"MLII" # pour ventricular et long term : 'ECG1'
        QRS = df["labels"].dropna().tolist()
        r_peaks = dico_algo[algo](signal, sampling_rate) #(signal, sampling_rate, 0)  # M, 
        #full_emd(signal, M, sampling_rate) ###
        perf_temp = perf(QRS, r_peaks, 36)
        Result_DF = pd.concat([Result_DF, pd.DataFrame({"data" : fichier[:-4], "TP": perf_temp[0], "FP": perf_temp[1], "FN":perf_temp[2]}, index=[0])], ignore_index=True)
    print(Result_DF)
    Result_DF["Sensi"] = Result_DF["TP"] / (Result_DF["TP"] + Result_DF["FN"])
    Result_DF["Specificity"] = Result_DF["TP"] / (Result_DF["TP"] + Result_DF["FP"])
    Result_DF["f1_score"] = (2 * Result_DF["TP"]) / (2 * Result_DF["TP"] + Result_DF["FN"] + Result_DF["FP"])
    Result_DF.sort_values("f1_score", inplace = True)
    end = time.time()
    print(f"{data}")
    print(f"f1_score = {(np.mean(Result_DF['f1_score'].tolist()) * 100).round(2)}% / Sensi = {(np.mean(Result_DF['Sensi'].tolist())* 100).round(2)}% / Spec = {(np.mean(Result_DF['Specificity'].tolist())* 100).round(2)}%  //// {round((end-start), 0)} seconds")

   data    TP   FP    FN
0   800  1804   79    89
1   801   503   73  1988
2   802  1650   25    18
3   803   649   53  1410
4   804   310    8  2514
..  ...   ...  ...   ...
73  890  1177  418   897
74  891  2222  187   221
75  892  1436   50  1387
76  893   464   24  2006
77  894  1918    7   440

[78 rows x 4 columns]
mit_bih_supraventricular
f1_score = 72.96% / Sensi = 66.51% / Spec = 93.2%  //// 141.0 seconds
     data     TP    FP    FN
0   e0103   3628    46  3703
1   e0104   6795   141   913
2   e0105   6455  1647   248
3   e0106   6654   864   510
4   e0108   6433   259   171
5   e0110   6829   683   264
6   e0111   2079   251  5461
7   e0112   2467   961  3126
8   e0113   8857    36   313
9   e0114    204     9  5394
10  e0115  10667    64   695
11  e0116   2873  1263  1656
12  e0118    113   198  7106
13  e0119    152   320  7921
14  e0121   6944   194  3807
15  e0122   5362    33  6046
16  e0123   4545  1914  4659
17  e0124   4021  1932  5273
18  e0125   4296    41  4838
19

In [ ]:
# wavelet 36:
mit_bih_Arrhythmia
f1_score = 97.34% / Sensi = 96.78% / Spec = 98.09%  //// 34.0 seconds
mit_bih_long_term
f1_score = 92.96% / Sensi = 87.39% / Spec = 99.75%  //// 266.0 seconds
mit_bih_noise_stress
f1_score = 88.49% / Sensi = 92.78% / Spec = 85.07%  //// 15.0 seconds
ventricular : 
f1_score = 95.77% / Sensi = 92.76% / Spec = 99.49%  //// 16.0 seconds
european
f1_score = 94.4% / Sensi = 95.57% / Spec = 94.21%  //// 129.0 seconds

In [ ]:
# hamilton 36
mit_bih_Arrhythmia
f1_score = 97.73% / Sensi = 95.89% / Spec = 99.84%  //// 72.0 seconds
dataset = mit_bih_long_term
f1_score = 81.97% / Sensi = 71.93% / Spec = 99.93%  //// 1195.0 seconds
dataset = mit_bih_noise_stress
f1_score = 88.59% / Sensi = 89.25% / Spec = 88.1%  //// 20.0 seconds
Ventricular :
f1_score = 89.9% / Sensi = 84.18% / Spec = 99.37%  //// 53.0 seconds
european
f1_score = 95.88% / Sensi = 94.02% / Spec = 98.1%  //// 329.0 seconds

# hamilton : 18
mit_bih_Arrhythmia
f1_score = 96.46% / Sensi = 94.69% / Spec = 98.5%  //// 68.0 seconds
mit_bih_long_term
f1_score = 73.66% / Sensi = 64.03% / Spec = 91.17%  //// 1391.0 seconds
mit_bih_noise_stress
f1_score = 86.84% / Sensi = 87.46% / Spec = 86.39%  //// 20.0 seconds
mit_bih_supraventricular
f1_score = 88.44% / Sensi = 82.72% / Spec = 97.93%  //// 53.0 seconds
european
f1_score = 86.16% / Sensi = 84.58% / Spec = 87.98%  //// 333.0 seconds

In [ ]:
# emd:
mit_bih_Arrhythmia
f1_score = 0.78% / Sensi = 5.15% / Spec = 0.68%  //// 860.0 seconds
noise:
f1_score = 0.76148 / Sensi = 0.75351 / Spec = 0.79285   /// 183.1329312324524
ventricular : 
f1_score = 0.79018 / Sensi = 0.71223 / Spec = 0.98957   /// 336.951664686203


In [15]:
def perf(labels, peaks, minmax):
    x = np.concatenate([np.array(labels), np.array(peaks)]) #list(set(QRS + r_peaks))
    x.sort()
    diff = x[1:]-x[:-1]
    gps = np.concatenate([[0], np.cumsum(diff>=minmax)])
    temp = [x[gps==i] for i in range(gps[-1]+1)]
    TP = 0
    FP = 0
    FN = 0
    list_F = []
    for sublist in temp:
        if len(sublist) == 2:
            TP += 1
        else:
            list_F.append(sublist)
            if sublist[0] in labels:
                FN += 1
            else:
                FP += 1
    return TP, FP, FN, list_F

In [19]:
def low_comput(ecg_signal, fs):
    k = fs//50
    filtered_signal = apply_fir_filter(ecg_signal, k)
    squared_signal = filtered_signal ** 2
    QRS_width = int(0.12 * fs)                                                    # 0.12 * fs
    smoothed_signal = moving_average(squared_signal, QRS_width)
    normalized_signal = normalize_operation(smoothed_signal)
    thv = threshold(normalized_signal, QRS_width)
    r_peaks_01 = [1 if thv[i] <= normalized_signal[i] else 0 for i in range(len(thv))]
    r_peaks = [i for i in range(len(r_peaks_01)) if r_peaks_01[i] == 1]
    r_peaks = regroup_nul(np.array(r_peaks), 36)
    Rsi = int(0.24 * fs)
    a = - Rsi
    tab = []

    for value in r_peaks:
        if value >= a + Rsi:
            tab.append(value)
            a = value
            
    r_peaks_real = super_filtre(ecg_signal, tab, Rsi)
    return r_peaks_real


def super_filtre(ecg_signal, r_peaks, Rsi):
    WQRS = Rsi-1 if Rsi%2 == 0 else Rsi         #(12)
    H_WQRS = WQRS//2
    list_peak = []
    L = 10
    for idx, peak in enumerate(r_peaks[10:]):
        sous_tab = r_peaks[idx-L:idx+L]
        
        direction = ( (1/(2*L)) * sum([ecg_signal[aaa] for aaa in sous_tab]) ) #13.a
        if list_peak == []:
            for p in r_peaks[:10]:
                list_peak.append(choose_max(ecg_signal, H_WQRS, p, -1))
                continue
        list_peak.append(choose_max(ecg_signal, H_WQRS, peak, direction))
    
    return list_peak

def threshold(signal, QRS_width):
    N = QRS_width
    thv = [1]
    for i in signal:
        thv.append(((N-1) * thv[-1] + i )/ N)
    return thv[1:]

def regroup_nul(peaks, thr):
    diff = peaks[1:]-peaks[:-1]
    gps = np.concatenate([[0], np.cumsum(diff>=thr)])
    temp = [peaks[gps==i] for i in range(gps[-1]+1)]
    return [np.mean(sublist).astype(int) for sublist in temp]

def choose_max(ecg_signal, H_WQRS, peak, direction):
    if direction > 0:
        return peak - H_WQRS + np.argmax(ecg_signal[peak-(H_WQRS):peak+(H_WQRS)])
    else:
        return peak - H_WQRS + np.argmin(ecg_signal[peak-(H_WQRS):peak+(H_WQRS)])
    
def apply_fir_filter(signal, k):
    y0 = np.zeros_like(signal)
    y0[k:] = signal[k:] - signal[:-k]
    return y0

def moving_average(signal, window_size):
    return np.convolve(signal, np.ones(window_size)/window_size, mode='same')

def normalize_operation(signal):
    return (signal - np.min(signal)) / (np.max(signal) - np.min(signal))

In [10]:
def full_emd(signal, sampling_rate):
    signal_without_baseline =  baseline_wander_cancel(signal, sampling_rate)
    l = []
    imfs = emd.sift.sift(signal_without_baseline)
    for i in range(0, 3):
        imf = [t[i] for t in imfs]    
        transformed_signal = nonlinear_transform(imf)
        transformed_signal = normalize_data(transformed_signal)
        M = 10
        integrated_signal = integration(transformed_signal, M)
        l.append(integrated_signal)
    arr = np.array(l)
    signal_end = arr.sum(axis=0)
    filtered_signal, r_peaks = low_pass_filter_and_r_position(signal_end , sampling_rate, 0.3, 400)
    return r_peaks

def recheck(r_peaks, signal, fs):
    rr_interval = np.diff(r_peaks)
    #search_interval = int(v * np.mean(rr_interval))
    missed_peaks = []
    w = int(fs*0.2)
    
    for i in range(1, len(r_peaks) - 1):
        if r_peaks[i+1] - r_peaks[i] > rr_interval[i-1] * 2:
            intervening_segment = signal[r_peaks[i]+w:r_peaks[i+1]-w]
            threshold = np.max(intervening_segment) * 0.3
            #a = qrs_localization_normal(ecg_signal, (max(ecg_signal)*0.3*0.5)
            a = qrs_localization(intervening_segment, threshold, fs)
            missed_peaks.extend([r_peaks[i] + x for x in a])
        missed_peaks.append(r_peaks[-1])
    return missed_peaks


def baseline_wander_cancel(ecg_signal, sampling_rate):
    cutoff_freq = 1  # Fréquence de coupure à 1 Hz
    order = 5  # Ordre du filtre
    b, a = scipy.signal.butter(order, 
                         cutoff_freq / (sampling_rate / 2), 
                         'highpass', 
                         analog=False)
    ecghp = scipy.signal.lfilter(b, a, ecg_signal)
    return ecghp

def low_pass_filter_and_r_position(signal, sampling_rate, seuil, w):
    cutoff = 50
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff / nyquist
    b, a = butter(1, normal_cutoff, btype='low', analog=False)
    filtered_signal = filtfilt(b, a, signal)

    derivative = np.diff(filtered_signal)
    squared = derivative ** 2
    #squared = np.clip(squared, a_min=None, a_max=0.0001)
    t = np.mean(squared)
    #print_signal(squared[:20000], "squared", print_qrs = False)
    r_peaks = qrs_localization(squared, np.mean(squared), sampling_rate) # np.mean(squared)) 
    return squared, r_peaks

def nonlinear_transform(x):
    y = [0] * len(x)
    for n in range(2, len(x)):
        if (x[n] >= 0 and x[n-1] >= 0 and x[n-2] >= 0) or (x[n] <= 0 and x[n-1] <= 0 and x[n-2] <= 0):
            y[n] = abs(x[n] * x[n-1] * x[n-2])
    return y

def integration(data, window_size):
    result = np.cumsum(data)
    result[window_size:] = result[window_size:] - result[:-window_size]
    return result[window_size - 1:] / window_size

def qrs_localization(h, XX, fs):
    qrs_indices = []
    sous_groupe = []
    #w = 1000
    threshold = XX # * max(h[:w])
    A = int(fs//10)
    compteur = 0
    for i in range(len(h)):
        if compteur != 0:
            compteur -= 1
            continue
        #if i%w == 0 and i != 0:
        #    threshold = XX * max(h[i-(w//2):i+(w//2)])
        if h[i] >= threshold:
            sous_groupe.append(i)
        elif sous_groupe != [] and i - sous_groupe[-1] >= A: # 36 = valeur donnée dans l'article
            v = max([(x, h[x]) for x in sous_groupe], key=lambda x: x[1])[0]
            qrs_indices.append(v)
            compteur = A
            sous_groupe = []
    if sous_groupe != []:
        qrs_indices.append(max([(x, h[x]) for x in sous_groupe], key=lambda x: x[1])[0])
    return qrs_indices

def normalize_data(signal):
    max_value = 0.7 * np.max(signal)
    normalized_signal = np.where(signal >= max_value, 1, signal / max_value)
    return normalized_signal

In [8]:
import pywt
def wavelet_decomposition(ecg_signal, idx):
    coeffs = pywt.wavedec(ecg_signal, 'haar', level=idx)
    d4 = extend_list_with_averages(coeffs[-2])
    d4.insert(0,0)
    h = np.abs(np.multiply(d4, coeffs[-1]))
    return h

def qrs_localization(h, thresold):
    qrs_indices = []
    sous_groupe = []
    w = 1000
    threshold = 0.15 * max(h[:w])
    for i in range(len(h)):
        if i%w == 0 and i != 0:
            threshold = 0.15 * max(h[i-(w//2):i+(w//2)])
        #print(h[i])
        if h[i] >= threshold:
            sous_groupe.append(i)
        elif sous_groupe != [] and i - sous_groupe[-1] >= 36: # 36 = valeur donnée dans l'article
            qrs_indices.append(max([(x, h[x]) for x in sous_groupe], key=lambda x: x[1])[0])
            sous_groupe = []
    if sous_groupe != []:
        qrs_indices.append(max([(x, h[x]) for x in sous_groupe], key=lambda x: x[1])[0])
    return qrs_indices

def delete_contraction(r_peaks, fs):
    new_r_peaks = []
    i = 0
    while i < len(r_peaks) -1:
        new_r_peaks.append(r_peaks[i])
        if r_peaks[i] >= r_peaks[i+1] + int(fs/5): # 5 * (fs / 2 )200 correspond à un mouvement du coeur particulier après un battement
            i += 2
        else:
            i += 1
    return new_r_peaks
            
def searchback_missed_qrs(ecg_signal, filtered_indices, fs):
    rr_interval = np.mean(np.diff(filtered_indices))
    search_interval = int(1.5 * rr_interval)
    secondary_threshold = 0.5 * max(ecg_signal)
    missed_peaks = []
    for i in range(len(filtered_indices) - 1):
        if filtered_indices[i+1] - filtered_indices[i] > search_interval:
            intervening_segment = ecg_signal[filtered_indices[i]:filtered_indices[i+1]]
            missed_peaks.extend(qrs_localization(intervening_segment, secondary_threshold))
    return missed_peaks

def extend_list_with_averages(input_list):
    extended_list = [input_list[0]]
    length = len(input_list)

    for i in range(length - 1):
        current_element = input_list[i]
        next_element = input_list[i + 1]
        average = (current_element + next_element) / 2
        extended_list.append(average)
        extended_list.append(next_element)

    return extended_list

def full_wavelets(signal, fs, group):
    h = wavelet_decomposition(signal, 5)
    h = extend_list_with_averages(h)
    threshold = 0.3 * max(h)
    qrs = np.array(qrs_localization(h, threshold))
    qrs.sort()
    qrs = delete_contraction(qrs, sampling_rate)
    qrs = qrs + searchback_missed_qrs(h, qrs, fs)
    return qrs